# Master Your Money: Financial Calculations with Python

## 掌控你的财务：用 Python 做金融计算

---

**Author**: [@realei](https://github.com/realei)  
**Level**: Beginner to Intermediate  
**Time**: ~30 minutes

---

## Why Should You Care? / 为什么要学这个？

Before we dive into code, let me show you something that might shock you:

在开始写代码之前，让我展示一个可能让你震惊的事实：

In [1]:
# The Hidden Cost of a 30-Year Mortgage / 30年房贷的隐藏成本

home_price = 500000      # $500,000 home / 50万美元的房子
down_payment = 100000    # 20% down / 20% 首付
loan_amount = 400000     # Amount borrowed / 贷款金额
annual_rate = 0.065      # 6.5% interest rate / 年利率
years = 30               # 30-year mortgage / 30年期

# Simple calculation (we'll learn the proper way soon!)
monthly_rate = annual_rate / 12
months = years * 12
monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate)**months) / ((1 + monthly_rate)**months - 1)
total_paid = monthly_payment * months
total_interest = total_paid - loan_amount

print("🏠 30-Year Mortgage Reality Check / 30年房贷真相")
print("=" * 50)
print(f"Home Price (房价):          ${home_price:>12,.0f}")
print(f"Down Payment (首付):        ${down_payment:>12,.0f}")
print(f"Loan Amount (贷款金额):     ${loan_amount:>12,.0f}")
print(f"Monthly Payment (月供):     ${monthly_payment:>12,.2f}")
print()
print(f"Total Paid (总还款):        ${total_paid:>12,.0f}")
print(f"Total Interest (总利息):    ${total_interest:>12,.0f}  ⚠️")
print()
print(f"💡 You pay ${total_interest/loan_amount:.0%} EXTRA in interest!")
print(f"💡 利息占本金的 {total_interest/loan_amount:.0%}！相当于多买了大半套房！")

🏠 30-Year Mortgage Reality Check / 30年房贷真相
Home Price (房价):          $     500,000
Down Payment (首付):        $     100,000
Loan Amount (贷款金额):     $     400,000
Monthly Payment (月供):     $    2,528.27

Total Paid (总还款):        $     910,178
Total Interest (总利息):    $     510,178  ⚠️

💡 You pay $128% EXTRA in interest!
💡 利息占本金的 128%！相当于多买了大半套房！


**That's right!** On a typical 30-year mortgage, you might pay **more in interest than the original loan amount**.

**没错！** 30年房贷，你支付的利息可能**超过贷款本金**。

### What if you chose 15 years instead? / 如果选择15年呢？

In [ ]:
# Compare 30-year vs 15-year mortgage
def calculate_mortgage(principal, rate, years):
    monthly_rate = rate / 12
    months = years * 12
    payment = principal * (monthly_rate * (1 + monthly_rate)**months) / ((1 + monthly_rate)**months - 1)
    total = payment * months
    interest = total - principal
    return payment, total, interest

pmt_30, total_30, interest_30 = calculate_mortgage(loan_amount, annual_rate, 30)
pmt_15, total_15, interest_15 = calculate_mortgage(loan_amount, annual_rate, 15)

print("📊 30-Year vs 15-Year Comparison / 30年 vs 15年对比")
print("=" * 55)
print(f"{'':20} {'30-Year':>15} {'15-Year':>15}")
print("-" * 55)
print(f"{'Monthly Payment':20} ${pmt_30:>14,.0f} ${pmt_15:>14,.0f}")
print(f"{'Total Interest':20} ${interest_30:>14,.0f} ${interest_15:>14,.0f}")
print()
print(f"🎯 Choosing 15 years saves you: ${interest_30 - interest_15:,.0f}")
print(f"🎯 选择15年可以省下: ${interest_30 - interest_15:,.0f}")
print(f"   (That's a {(interest_30 - interest_15)/interest_30:.0%} reduction in interest!)")

---

## What You'll Learn / 你将学到

This notebook will teach you to:

1. **Calculate loan payments** - How much will I pay each month?
2. **Understand amortization** - Where does my money go?
3. **Make informed decisions** - Which loan term is best for me?
4. **Build financial planning tools** - Your personal finance assistant

这个 notebook 将教你：

1. **计算贷款月供** - 每月要还多少？
2. **理解还款结构** - 我的钱去哪了？
3. **做出明智决策** - 哪种贷款方案最适合我？
4. **构建财务规划工具** - 你的个人财务助手

---

# Part 1: The Basics / 基础篇

## Setup / 环境准备

This notebook works in both **local** (VS Code) and **Kaggle** environments.

In [ ]:
# ============================================================
# Environment Detection / 环境检测
# ============================================================
import os
import sys

# Detect environment
IS_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IS_COLAB = "COLAB_GPU" in os.environ or "google.colab" in sys.modules
ENV_NAME = "kaggle" if IS_KAGGLE else ("colab" if IS_COLAB else "local")

print(f"🖥️  Environment: {ENV_NAME.upper()}")

# For local development: add src to path for imports
if ENV_NAME == "local":
    # Add parent directory to path (where src/ is located)
    project_root = os.path.dirname(os.getcwd())
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    print(f"   Project root: {project_root}")

# Install numpy-financial if needed (mainly for Kaggle/Colab)
try:
    import numpy_financial as npf
except ImportError:
    print("Installing numpy-financial...")
    !pip install -q numpy-financial
    import numpy_financial as npf

# ============================================================
# Import Libraries / 导入库
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Make plots look nice
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

print(f"✅ All libraries loaded!")
print(f"   numpy_financial version: {npf.__version__}")

## Why numpy_financial? / 为什么用 numpy_financial?

Remember that ugly formula we used earlier? numpy_financial makes it simple:

还记得刚才那个复杂的公式吗？numpy_financial 让它变得简单：

```python
# Before (手写公式):
payment = principal * (rate * (1 + rate)**n) / ((1 + rate)**n - 1)

# After (用 numpy_financial):
payment = -npf.pmt(rate, n, principal)
```

**Benefits / 优点:**
- Same results as Excel's PMT function / 与 Excel PMT 函数结果一致
- Handles edge cases (0% interest, etc.) / 处理边界情况
- Much faster for batch calculations / 批量计算更快

## Core Function #1: pmt() - Monthly Payment / 月供计算

The most commonly used function. Calculates regular payment amount.

最常用的函数，计算每期还款金额。

In [ ]:
# Example: Car loan / 例子：车贷
car_price = 30000        # $30,000 car
down_payment = 5000      # $5,000 down
loan = car_price - down_payment
rate = 0.059             # 5.9% APR
term = 60                # 5 years = 60 months

# Calculate monthly payment
monthly_rate = rate / 12
payment = -npf.pmt(monthly_rate, term, loan)  # Note: we use negative to get positive result

print("🚗 Car Loan Calculator / 车贷计算器")
print("=" * 40)
print(f"Car Price (车价):      ${car_price:>10,}")
print(f"Down Payment (首付):   ${down_payment:>10,}")
print(f"Loan Amount (贷款):    ${loan:>10,}")
print(f"Interest Rate (利率):  {rate:>10.1%}")
print(f"Term (期限):           {term:>10} months")
print()
print(f"📌 Monthly Payment:    ${payment:>10,.2f}")
print(f"📌 Total Cost:         ${payment * term:>10,.2f}")
print(f"📌 Total Interest:     ${payment * term - loan:>10,.2f}")

### Understanding Sign Conventions / 理解正负号

numpy_financial uses **cash flow conventions**:
- **Negative** = Money going OUT (payments you make)
- **Positive** = Money coming IN (loan you receive)

numpy_financial 使用**现金流约定**：
- **负数** = 钱流出（你付出的）
- **正数** = 钱流入（你收到的）

In [ ]:
# Demonstration of sign conventions
print("Sign Convention Demo / 正负号演示")
print("=" * 45)

raw_result = npf.pmt(monthly_rate, term, loan)
print(f"npf.pmt() returns: {raw_result:.2f}")
print(f"  → Negative because it's money you PAY out")
print()
print(f"We use -npf.pmt() = {-raw_result:.2f}")
print(f"  → Positive for easier reading")
print()
print("💡 Tip: Always use negative (-) when displaying payments")
print("💡 技巧：显示月供时记得加负号")

## Core Function #2: pv() - How Much Can I Borrow? / 我能借多少？

The **reverse** of pmt(). Given a payment you can afford, how much can you borrow?

pmt() 的**逆运算**。给定你能承受的月供，能借多少钱？

In [ ]:
# I can afford $500/month, how much car can I buy?
affordable_payment = 500
rate = 0.059
term = 60

max_loan = -npf.pv(rate/12, term, affordable_payment)

print("💰 Affordability Calculator / 负担能力计算器")
print("=" * 45)
print(f"I can afford (我能承受): ${affordable_payment}/month")
print(f"Interest rate (利率):    {rate:.1%}")
print(f"Loan term (期限):        {term} months")
print()
print(f"📌 Maximum I can borrow: ${max_loan:,.2f}")
print(f"📌 我最多能贷: ${max_loan:,.2f}")

---

# Part 2: Where Does My Money Go? / 我的钱去哪了？

## Understanding Amortization / 理解还款分期

Each payment is split into two parts:
1. **Interest** - The bank's profit / 银行的利润
2. **Principal** - Actually paying down your debt / 真正减少你的债务

每次还款分为两部分：
1. **利息** - 银行赚的钱
2. **本金** - 真正减少债务的部分

In [ ]:
# Let's see how a $25,000 loan breaks down
loan = 25000
rate = 0.059
term = 60
monthly_rate = rate / 12

payment = -npf.pmt(monthly_rate, term, loan)

# First month vs Last month comparison
# ipmt = interest payment, ppmt = principal payment
first_interest = -npf.ipmt(monthly_rate, 1, term, loan)
first_principal = -npf.ppmt(monthly_rate, 1, term, loan)

last_interest = -npf.ipmt(monthly_rate, term, term, loan)
last_principal = -npf.ppmt(monthly_rate, term, term, loan)

print("📊 Payment Breakdown / 还款构成分析")
print("=" * 55)
print(f"Loan: ${loan:,} at {rate:.1%} for {term} months")
print(f"Monthly Payment: ${payment:.2f}")
print()
print(f"{'':15} {'Interest':>12} {'Principal':>12} {'Total':>12}")
print("-" * 55)
print(f"{'Month 1 (首月)':15} ${first_interest:>11.2f} ${first_principal:>11.2f} ${payment:>11.2f}")
print(f"{'Month 60 (末月)':15} ${last_interest:>11.2f} ${last_principal:>11.2f} ${payment:>11.2f}")
print()
print("💡 Notice: Same total payment, but allocation changes dramatically!")
print("💡 注意：总月供相同，但利息/本金比例变化巨大！")

## Visualizing the Amortization / 可视化还款过程

Let's see how interest vs principal changes over time.

In [ ]:
# Generate full amortization schedule
months = np.arange(1, term + 1)
interest_payments = -npf.ipmt(monthly_rate, months, term, loan)
principal_payments = -npf.ppmt(monthly_rate, months, term, loan)
balance = loan - np.cumsum(principal_payments)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Stacked area chart
ax1 = axes[0]
ax1.stackplot(months, principal_payments, interest_payments,
              labels=['Principal', 'Interest'],
              colors=['#27ae60', '#e74c3c'], alpha=0.8)
ax1.axhline(y=payment, color='navy', linestyle='--', linewidth=1.5, label=f'Total Payment: ${payment:.0f}')
ax1.set_xlabel('Month')
ax1.set_ylabel('Payment Amount ($)')
ax1.set_title('How Your Payment Is Split Each Month')
ax1.legend(loc='right')
ax1.set_xlim(1, term)

# Right plot: Balance over time
ax2 = axes[1]
ax2.fill_between(months, balance, color='#3498db', alpha=0.5)
ax2.plot(months, balance, color='#2980b9', linewidth=2)

# Add milestone markers
halfway = term // 2
halfway_balance = balance[halfway - 1]
ax2.scatter([halfway], [halfway_balance], color='orange', s=100, zorder=5)
ax2.annotate(f'Halfway: ${halfway_balance:,.0f}\n({halfway_balance/loan:.0%} remaining)', 
             xy=(halfway, halfway_balance),
             xytext=(halfway + 5, halfway_balance + 3000),
             fontsize=10,
             arrowprops=dict(arrowstyle='->', color='orange'))

ax2.set_xlabel('Month')
ax2.set_ylabel('Remaining Balance ($)')
ax2.set_title('Loan Balance Over Time')
ax2.set_xlim(1, term)
ax2.set_ylim(0, loan * 1.1)

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight: At month {halfway}, you've made {halfway} payments but still owe {halfway_balance/loan:.0%} of the loan!")
print(f"💡 关键洞察：第 {halfway} 个月时，你已还了一半期数，但还欠 {halfway_balance/loan:.0%} 的本金！")

---

# Part 3: Personal Finance Planning Tools / 个人财务规划工具

Now let's build practical tools you can use for your own financial decisions.

现在让我们构建你可以实际使用的财务规划工具。

## Tool 1: 🏠 Home Buying Planner / 购房规划器

Figure out what you can afford based on your income.

In [ ]:
def home_buying_planner(monthly_income, existing_debt_payment=0, 
                        down_payment_saved=0, interest_rate=0.065,
                        loan_term_years=30, max_dti=0.36):
    """
    Calculate how much home you can afford.
    计算你能买多贵的房子。
    
    Args:
        monthly_income: Your gross monthly income / 月收入（税前）
        existing_debt_payment: Current monthly debt payments / 现有月供（信用卡、车贷等）
        down_payment_saved: Amount saved for down payment / 已攒首付
        interest_rate: Expected mortgage rate / 预期房贷利率
        loan_term_years: Loan term in years / 贷款年限
        max_dti: Maximum debt-to-income ratio / 最大负债收入比
    """
    # Calculate max monthly payment (industry standard: 36% DTI)
    max_total_debt = monthly_income * max_dti
    max_mortgage_payment = max_total_debt - existing_debt_payment
    
    # Calculate max loan amount
    monthly_rate = interest_rate / 12
    months = loan_term_years * 12
    max_loan = -npf.pv(monthly_rate, months, max_mortgage_payment)
    
    # Estimate max home price (assuming 20% down)
    max_home_price = max_loan / 0.8
    required_down = max_home_price * 0.2
    
    # Calculate total costs
    total_paid = max_mortgage_payment * months
    total_interest = total_paid - max_loan
    
    print("🏠 Home Buying Analysis / 购房能力分析")
    print("=" * 55)
    print(f"\n📋 Your Inputs / 你的输入:")
    print(f"   Monthly Income (月收入):     ${monthly_income:>12,}")
    print(f"   Existing Debts (现有月供):   ${existing_debt_payment:>12,}")
    print(f"   Down Payment (已攒首付):     ${down_payment_saved:>12,}")
    print(f"   Interest Rate (利率):        {interest_rate:>12.2%}")
    print(f"   Loan Term (贷款年限):        {loan_term_years:>12} years")
    
    print(f"\n📊 Results / 分析结果:")
    print(f"   Max Monthly Payment (最高月供): ${max_mortgage_payment:>10,.0f}")
    print(f"   Max Loan Amount (最高贷款):     ${max_loan:>10,.0f}")
    print(f"   Max Home Price (最高房价):      ${max_home_price:>10,.0f}")
    print(f"   Required Down (需要首付):       ${required_down:>10,.0f}")
    
    print(f"\n💰 Total Cost Analysis / 总成本分析:")
    print(f"   Total Paid Over {loan_term_years} Years:      ${total_paid:>10,.0f}")
    print(f"   Total Interest (总利息):        ${total_interest:>10,.0f}")
    
    # Status check
    print(f"\n{'✅' if down_payment_saved >= required_down else '⚠️'} Down Payment Status:")
    if down_payment_saved >= required_down:
        print(f"   You have enough! Surplus: ${down_payment_saved - required_down:,.0f}")
    else:
        print(f"   Need ${required_down - down_payment_saved:,.0f} more for 20% down")
        print(f"   还需要攒 ${required_down - down_payment_saved:,.0f}")
    
    return {
        'max_payment': max_mortgage_payment,
        'max_loan': max_loan,
        'max_home_price': max_home_price,
        'required_down': required_down,
        'total_interest': total_interest
    }

In [ ]:
# Example: A person earning $8,000/month with $500 car payment
# 例子：月收入 $8,000，有 $500 车贷

result = home_buying_planner(
    monthly_income=8000,
    existing_debt_payment=500,
    down_payment_saved=60000,
    interest_rate=0.065,
    loan_term_years=30
)

## Tool 2: 🚗 Car Loan Comparison / 车贷比较器

Compare different loan terms to make the best decision.

In [ ]:
def car_loan_comparison(car_price, down_payment, interest_rate, 
                        terms=[36, 48, 60, 72]):
    """
    Compare different car loan terms.
    比较不同车贷期限。
    """
    loan_amount = car_price - down_payment
    monthly_rate = interest_rate / 12
    
    results = []
    for term in terms:
        payment = -npf.pmt(monthly_rate, term, loan_amount)
        total = payment * term
        interest = total - loan_amount
        results.append({
            'Term (months)': term,
            'Term (years)': f"{term/12:.1f} yrs",
            'Monthly Payment': payment,
            'Total Paid': total,
            'Total Interest': interest,
            'Interest %': interest / loan_amount * 100
        })
    
    df = pd.DataFrame(results)
    
    print("🚗 Car Loan Comparison / 车贷方案对比")
    print("=" * 65)
    print(f"Car Price: ${car_price:,} | Down Payment: ${down_payment:,} | Loan: ${loan_amount:,}")
    print(f"Interest Rate: {interest_rate:.2%}")
    print()
    
    # Format and display
    styled_df = df.style.format({
        'Monthly Payment': '${:,.0f}',
        'Total Paid': '${:,.0f}',
        'Total Interest': '${:,.0f}',
        'Interest %': '{:.1f}%'
    })
    display(styled_df)
    
    # Recommendation
    min_interest = df.loc[df['Total Interest'].idxmin()]
    print(f"\n💡 Recommendation: {min_interest['Term (months)']} months saves the most interest!")
    print(f"   But monthly payment is ${min_interest['Monthly Payment']:,.0f}")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 5))
    x = np.arange(len(terms))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, df['Monthly Payment'], width, label='Monthly Payment', color='#3498db')
    ax2 = ax.twinx()
    bars2 = ax2.bar(x + width/2, df['Total Interest'], width, label='Total Interest', color='#e74c3c', alpha=0.7)
    
    ax.set_xlabel('Loan Term')
    ax.set_ylabel('Monthly Payment ($)', color='#3498db')
    ax2.set_ylabel('Total Interest ($)', color='#e74c3c')
    ax.set_xticks(x)
    ax.set_xticklabels([f"{t} mo\n({t/12:.1f} yr)" for t in terms])
    ax.set_title('Monthly Payment vs Total Interest by Term')
    
    # Add value labels
    for bar in bars1:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                f'${bar.get_height():,.0f}', ha='center', va='bottom', fontsize=9)
    for bar in bars2:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                 f'${bar.get_height():,.0f}', ha='center', va='bottom', fontsize=9, color='#e74c3c')
    
    fig.legend(loc='upper right', bbox_to_anchor=(0.88, 0.88))
    plt.tight_layout()
    plt.show()
    
    return df

In [ ]:
# Example: $35,000 car with $5,000 down at 6.5% rate
comparison = car_loan_comparison(
    car_price=35000,
    down_payment=5000,
    interest_rate=0.065
)

## Tool 3: 💰 Early Payoff Calculator / 提前还款计算器

See how much you can save by paying extra each month.

In [ ]:
def early_payoff_calculator(loan_amount, interest_rate, term_months, extra_payment=0):
    """
    Calculate savings from making extra payments.
    计算提前还款能省多少钱。
    
    Args:
        loan_amount: Original loan amount
        interest_rate: Annual interest rate
        term_months: Original loan term in months
        extra_payment: Extra amount paid each month
    """
    monthly_rate = interest_rate / 12
    base_payment = -npf.pmt(monthly_rate, term_months, loan_amount)
    total_payment = base_payment + extra_payment
    
    # Simulate payments with extra
    balance = loan_amount
    months_paid = 0
    total_interest_paid = 0
    
    while balance > 0 and months_paid < term_months:
        months_paid += 1
        interest = balance * monthly_rate
        total_interest_paid += interest
        principal = min(total_payment - interest, balance)
        balance = max(0, balance - principal)
    
    # Calculate original scenario
    original_total_interest = base_payment * term_months - loan_amount
    
    # Results
    months_saved = term_months - months_paid
    interest_saved = original_total_interest - total_interest_paid
    
    print("💰 Early Payoff Analysis / 提前还款分析")
    print("=" * 55)
    print(f"Loan: ${loan_amount:,} at {interest_rate:.2%} for {term_months} months")
    print(f"Base Payment: ${base_payment:,.2f}/month")
    print(f"Extra Payment: ${extra_payment:,.2f}/month")
    print()
    
    print(f"{'':25} {'Original':>12} {'With Extra':>12}")
    print("-" * 55)
    print(f"{'Monthly Payment':25} ${base_payment:>11,.0f} ${total_payment:>11,.0f}")
    print(f"{'Months to Payoff':25} {term_months:>12} {months_paid:>12}")
    print(f"{'Total Interest':25} ${original_total_interest:>11,.0f} ${total_interest_paid:>11,.0f}")
    print()
    print(f"🎉 You save {months_saved} months ({months_saved/12:.1f} years)!")
    print(f"🎉 You save ${interest_saved:,.0f} in interest!")
    print(f"🎉 提前 {months_saved} 个月还清，省下 ${interest_saved:,.0f} 利息！")
    
    return {
        'months_saved': months_saved,
        'interest_saved': interest_saved,
        'new_payoff_months': months_paid
    }

In [ ]:
# Example: $200,000 mortgage, pay $300 extra/month
savings = early_payoff_calculator(
    loan_amount=200000,
    interest_rate=0.065,
    term_months=360,  # 30 years
    extra_payment=300
)

## Tool 4: 🎯 Savings Goal Calculator / 储蓄目标计算器

How much do you need to save monthly to reach a goal?

In [ ]:
def savings_goal_calculator(goal_amount, years, annual_return=0.07, 
                            current_savings=0):
    """
    Calculate monthly savings needed to reach a goal.
    计算达到目标需要每月存多少钱。
    
    Args:
        goal_amount: Target amount to save
        years: Years until goal
        annual_return: Expected annual return (default 7%)
        current_savings: Amount already saved
    """
    monthly_rate = annual_return / 12
    months = years * 12
    
    # Future value of current savings
    fv_current = current_savings * (1 + monthly_rate) ** months
    
    # Remaining amount needed
    remaining_goal = goal_amount - fv_current
    
    # Monthly savings needed (using pmt with fv)
    if remaining_goal > 0:
        monthly_savings = -npf.pmt(monthly_rate, months, 0, remaining_goal)
    else:
        monthly_savings = 0
    
    # Total contributions
    total_contributed = current_savings + monthly_savings * months
    total_growth = goal_amount - total_contributed
    
    print("🎯 Savings Goal Calculator / 储蓄目标计算器")
    print("=" * 50)
    print(f"Goal (目标):           ${goal_amount:>12,}")
    print(f"Time Frame (时间):     {years:>12} years")
    print(f"Expected Return (回报): {annual_return:>11.1%}")
    print(f"Current Savings (现有): ${current_savings:>12,}")
    print()
    
    if monthly_savings > 0:
        print(f"📌 Save ${monthly_savings:,.0f}/month to reach your goal!")
        print(f"📌 每月存 ${monthly_savings:,.0f} 即可达成目标！")
        print()
        print(f"Your Contributions (你的投入): ${total_contributed:>10,.0f}")
        print(f"Investment Growth (投资增值):  ${total_growth:>10,.0f}")
        print(f"Growth Percentage (增值比例):  {total_growth/total_contributed:>10.0%}")
    else:
        print(f"🎉 Great news! Your current savings will grow to ${fv_current:,.0f}")
        print(f"   That's ${fv_current - goal_amount:,.0f} MORE than your goal!")
    
    return monthly_savings

In [ ]:
# Example 1: Save for down payment in 5 years
print("Example 1: Saving for a home down payment\n")
savings_goal_calculator(
    goal_amount=100000,
    years=5,
    annual_return=0.05,
    current_savings=20000
)

In [ ]:
# Example 2: Retirement savings
print("Example 2: Retirement savings (starting at 30, retiring at 65)\n")
savings_goal_calculator(
    goal_amount=1000000,
    years=35,
    annual_return=0.07,
    current_savings=10000
)

---

# Part 4: Interactive Dashboard / 交互式仪表板

Put it all together in a comprehensive loan analyzer.

In [ ]:
def loan_dashboard(principal, annual_rate, term_months, extra_payment=0):
    """
    Comprehensive loan analysis dashboard.
    综合贷款分析仪表板。
    """
    monthly_rate = annual_rate / 12
    payment = -npf.pmt(monthly_rate, term_months, principal)
    
    # Generate amortization schedule
    months = np.arange(1, term_months + 1)
    interest = -npf.ipmt(monthly_rate, months, term_months, principal)
    principal_paid = -npf.ppmt(monthly_rate, months, term_months, principal)
    balance = principal - np.cumsum(principal_paid)
    balance = np.maximum(0, balance)
    balance[-1] = 0
    
    # Cumulative calculations
    cum_interest = np.cumsum(interest)
    cum_principal = np.cumsum(principal_paid)
    
    total_interest = cum_interest[-1]
    total_paid = payment * term_months
    
    # Create figure
    fig = plt.figure(figsize=(16, 10))
    
    # Title
    fig.suptitle(f'Loan Analysis Dashboard\n'
                 f'${principal:,} at {annual_rate:.2%} for {term_months//12} years',
                 fontsize=16, fontweight='bold')
    
    # Plot 1: Payment breakdown over time
    ax1 = fig.add_subplot(2, 2, 1)
    ax1.stackplot(months, principal_paid, interest,
                  labels=['Principal', 'Interest'],
                  colors=['#27ae60', '#e74c3c'], alpha=0.8)
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Amount ($)')
    ax1.set_title('Monthly Payment Composition')
    ax1.legend(loc='upper right')
    ax1.set_xlim(1, term_months)
    
    # Plot 2: Balance and cumulative interest
    ax2 = fig.add_subplot(2, 2, 2)
    ax2.plot(months, balance, 'b-', linewidth=2, label='Remaining Balance')
    ax2.plot(months, cum_interest, 'r--', linewidth=2, label='Cumulative Interest')
    ax2.fill_between(months, balance, alpha=0.3)
    ax2.set_xlabel('Month')
    ax2.set_ylabel('Amount ($)')
    ax2.set_title('Balance vs Cumulative Interest')
    ax2.legend()
    ax2.set_xlim(1, term_months)
    
    # Plot 3: Pie chart of total costs
    ax3 = fig.add_subplot(2, 2, 3)
    sizes = [principal, total_interest]
    labels = [f'Principal\n${principal:,.0f}', f'Interest\n${total_interest:,.0f}']
    colors = ['#27ae60', '#e74c3c']
    explode = (0, 0.05)
    ax3.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    ax3.set_title(f'Total Cost Breakdown\nTotal: ${total_paid:,.0f}')
    
    # Plot 4: Summary stats
    ax4 = fig.add_subplot(2, 2, 4)
    ax4.axis('off')
    
    summary_text = f"""
    📊 LOAN SUMMARY
    {'─' * 40}
    
    Loan Amount:          ${principal:>12,}
    Interest Rate:        {annual_rate:>12.2%}
    Term:                 {term_months:>12} months
    
    Monthly Payment:      ${payment:>12,.2f}
    Total Payments:       ${total_paid:>12,.0f}
    Total Interest:       ${total_interest:>12,.0f}
    
    Interest/Principal:   {total_interest/principal:>12.1%}
    
    {'─' * 40}
    First Month: ${interest[0]:.0f} interest, ${principal_paid[0]:.0f} principal
    Last Month:  ${interest[-1]:.0f} interest, ${principal_paid[-1]:.0f} principal
    """
    
    ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes, 
             fontsize=11, fontfamily='monospace',
             verticalalignment='center')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()
    
    return pd.DataFrame({
        'Month': months,
        'Payment': payment,
        'Principal': principal_paid,
        'Interest': interest,
        'Balance': balance,
        'Cumulative Interest': cum_interest,
        'Cumulative Principal': cum_principal
    })

In [ ]:
# Analyze a 30-year mortgage
schedule = loan_dashboard(
    principal=300000,
    annual_rate=0.065,
    term_months=360
)

In [ ]:
# View amortization schedule (first and last 5 rows)
print("Amortization Schedule (First 5 months):")
display(schedule.head().style.format({
    'Payment': '${:,.2f}',
    'Principal': '${:,.2f}',
    'Interest': '${:,.2f}',
    'Balance': '${:,.2f}',
    'Cumulative Interest': '${:,.2f}',
    'Cumulative Principal': '${:,.2f}'
}))

print("\nAmortization Schedule (Last 5 months):")
display(schedule.tail().style.format({
    'Payment': '${:,.2f}',
    'Principal': '${:,.2f}',
    'Interest': '${:,.2f}',
    'Balance': '${:,.2f}',
    'Cumulative Interest': '${:,.2f}',
    'Cumulative Principal': '${:,.2f}'
}))

---

# Part 5: Advanced - Building a Reusable Engine / 进阶 - 构建可复用引擎

For those who want to take this further, here's how to wrap everything into a clean, testable class.

对于想要深入学习的人，这里展示如何将所有功能封装成一个干净、可测试的类。

In [ ]:
from typing import Protocol


class PaymentCalculator(Protocol):
    """Protocol for payment calculations - enables duck typing."""
    def payment(self, principal: float, rate: float, periods: int) -> float: ...
    def max_principal(self, payment: float, rate: float, periods: int) -> float: ...


class FinancialEngine:
    """
    Financial calculation engine wrapping numpy-financial.
    
    This class follows SOLID principles:
    - Single Responsibility: Only handles mathematical calculations
    - Open/Closed: Extend by adding methods, not modifying existing ones
    - Dependency Inversion: Other modules depend on this abstraction
    
    Example:
        >>> engine = FinancialEngine()
        >>> monthly = engine.payment(principal=50000, rate=0.05, periods=36)
        >>> print(f"Monthly payment: ${monthly:.2f}")
        Monthly payment: $1498.88
    """

    def payment(self, principal: float, rate: float, periods: int) -> float:
        """Calculate monthly payment (EMI)."""
        if rate == 0:
            return principal / periods
        monthly_rate = rate / 12
        return float(-npf.pmt(monthly_rate, periods, principal))

    def max_principal(self, payment: float, rate: float, periods: int) -> float:
        """Calculate maximum principal for a given monthly payment."""
        if rate == 0:
            return payment * periods
        monthly_rate = rate / 12
        return float(-npf.pv(monthly_rate, periods, payment))

    def interest_payment(self, principal: float, rate: float, period: int, periods: int) -> float:
        """Calculate interest portion for a specific period."""
        if rate == 0:
            return 0.0
        monthly_rate = rate / 12
        return float(-npf.ipmt(monthly_rate, period, periods, principal))

    def principal_payment(self, principal: float, rate: float, period: int, periods: int) -> float:
        """Calculate principal portion for a specific period."""
        if rate == 0:
            return principal / periods
        monthly_rate = rate / 12
        return float(-npf.ppmt(monthly_rate, period, periods, principal))

    def remaining_balance(self, principal: float, rate: float, period: int, periods: int) -> float:
        """Calculate remaining balance after a specific period."""
        if rate == 0:
            pmt = principal / periods
            return max(0.0, principal - pmt * period)
        monthly_rate = rate / 12
        per = np.arange(1, period + 1)
        principal_paid = -npf.ppmt(monthly_rate, per, periods, principal)
        return max(0.0, principal - float(np.sum(principal_paid)))

    def amortization_table(self, principal: float, rate: float, periods: int) -> pd.DataFrame:
        """Generate full amortization schedule using vectorized operations."""
        if rate == 0:
            pmt = principal / periods
            per = np.arange(1, periods + 1)
            return pd.DataFrame({
                "month": per, "payment": pmt, "principal": pmt,
                "interest": 0.0, "balance": np.maximum(0, principal - pmt * per)
            })
        
        monthly_rate = rate / 12
        pmt = float(-npf.pmt(monthly_rate, periods, principal))
        per = np.arange(1, periods + 1)
        interest = -npf.ipmt(monthly_rate, per, periods, principal)
        principal_paid = -npf.ppmt(monthly_rate, per, periods, principal)
        balance = np.maximum(0, principal - np.cumsum(principal_paid))
        balance[-1] = 0
        
        return pd.DataFrame({
            "month": per, "payment": pmt, "principal": principal_paid,
            "interest": interest, "balance": balance
        })


# Global instance
engine = FinancialEngine()

# Test the engine
print("✅ FinancialEngine created successfully!")
print(f"   Test: $50,000 at 5% for 36 months = ${engine.payment(50000, 0.05, 36):,.2f}/month")

---

# Summary / 总结

## What We Learned / 我们学到了什么

### Function Name Abbreviations / 函数名缩写

💡 **Understanding the abbreviations helps you remember what each function does!**  
💡 **理解缩写能帮你快速记住每个函数的用途！**

| Function | Full Name | Memory Tip / 记忆技巧 |
|----------|-----------|----------------------|
| **pmt()** | **P**ay**M**en**T** | Payment you make each period / 每期付款额 |
| **pv()** | **P**resent **V**alue | Present value = How much you can borrow NOW / 现值 = 现在能借多少 |
| **fv()** | **F**uture **V**alue | Future value = What it's worth LATER / 终值 = 以后值多少 |
| **ipmt()** | **I**nterest **P**ay**M**en**T** | **I** for Interest payment / **I** 代表利息 |
| **ppmt()** | **P**rincipal **P**ay**M**en**T** | **P** for Principal payment / **P** 代表本金 |
| **rate()** | Interest **Rate** | Calculates interest rate / 计算利率 |
| **nper()** | **N**umber of **PER**iods | Number of periods / 期数 |

**Quick Memory Trick / 快速记忆：**
- **i**pmt → **I**nterest (利息)
- **p**pmt → **P**rincipal (本金)
- **p**v → **P**resent (现在/现值)
- **f**v → **F**uture (未来/终值)

### Core numpy_financial Functions

| Function | Purpose | Example |
|----------|---------|----------|
| `npf.pmt()` | Monthly payment | `-npf.pmt(rate/12, months, principal)` |
| `npf.pv()` | Max loan for payment | `-npf.pv(rate/12, months, payment)` |
| `npf.ipmt()` | Interest portion | `-npf.ipmt(rate/12, period, months, principal)` |
| `npf.ppmt()` | Principal portion | `-npf.ppmt(rate/12, period, months, principal)` |

### Financial Planning Tools Built

1. 🏠 **Home Buying Planner** - How much home can you afford?
2. 🚗 **Car Loan Comparison** - Compare different loan terms
3. 💰 **Early Payoff Calculator** - Save money by paying extra
4. 🎯 **Savings Goal Calculator** - Plan for future goals
5. 📊 **Loan Dashboard** - Comprehensive visualization

### Key Insights / 关键洞察

1. **Shorter terms save money** - Higher payments, but much less interest
2. **Early payments have most impact** - Interest is front-loaded
3. **Small extra payments add up** - $300/month extra can save years
4. **Time is your friend for savings** - Start early, let compound interest work

---

## Next Steps / 下一步

- **Notebook 2**: NPV and IRR for Investment Analysis / 投资分析
- **Notebook 3**: Rate and NPER calculations / 利率和期数计算
- **Notebook 4**: Building a complete personal finance app / 完整财务应用

---

**If this helped you, please upvote! 👍**

**如果对你有帮助，请点赞！**

Questions? Leave a comment below! / 有问题请留言！